In [ ]:
!pip install tabpfn --no-index --find-links=file:///kaggle/input/pip-packages-icr/pip-packages

In [ ]:
!mkdir -p /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
!cp /kaggle/input/pip-packages-icr/pip-packages/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

In [1]:
import numpy as np
import pandas as pd
import json
from sklearn.base import BaseEstimator
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
from catboost import Pool, CatBoostClassifier
import xgboost
from tabpfn import TabPFNClassifier

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings("ignore")

In [2]:
from pyprojroot import here

In [3]:
here()

WindowsPath('c:/Users/c.gendron1/Git/icr-identify-age-related-conditions-kaggle')

In [4]:

train_file = here('icr-identify-age-related-conditions/train.csv')
greeks_file = here('icr-identify-age-related-conditions/greeks.csv')
test_file = here('icr-identify-age-related-conditions/test.csv')

In [5]:
train_df = pd.read_csv(train_file)
greeks_df = pd.read_csv(greeks_file)
test_df = pd.read_csv(test_file)

In [6]:
print(train_df.EJ.unique())

['B' 'A']


In [7]:
first_category = train_df.EJ.unique()[0]
train_df.EJ = train_df.EJ.eq(first_category).astype('int')
test_df.EJ = test_df.EJ.eq(first_category).astype('int')

In [8]:
from datetime import datetime
times = greeks_df.Epsilon.copy()
times[greeks_df.Epsilon != 'Unknown'] = greeks_df.Epsilon[greeks_df.Epsilon != 'Unknown'].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())
times[greeks_df.Epsilon == 'Unknown'] = np.nan

In [9]:
target_column = 'Class'
predictor_columns = [n for n in train_df.columns if n != target_column and n != 'Id']

In [14]:
class WeightedEnsemble(BaseEstimator):
    def __init__(self):
        self.classifiers = [xgboost.XGBClassifier(), TabPFNClassifier(N_ensemble_configurations=64)]
        self.imputer = SimpleImputer(missing_values=np.nan, strategy='median')
    
    def fit(self, X, y):
        unique_classes, y = np.unique(y, return_inverse=True)
        self.classes_ = unique_classes
        X = self.imputer.fit_transform(X)
        for classifier in self.classifiers:
            classifier.fit(X, y)
    
    def predict_proba(self, X):
        X = self.imputer.transform(X)
        probabilities = np.stack([classifier.predict_proba(X) for classifier in self.classifiers])
        averaged_probabilities = np.mean(probabilities, axis=0)
        class_0_est_instances = averaged_probabilities[:, 0].sum()
        others_est_instances = averaged_probabilities[:, 1:].sum()
        # Weighted probabilities based on class imbalance
        new_probabilities = averaged_probabilities * np.array([[1/(class_0_est_instances if i==0 else others_est_instances) for i in range(averaged_probabilities.shape[1])]])
        return new_probabilities / np.sum(new_probabilities, axis=1, keepdims=1)


In [15]:
train_pred_and_time = pd.concat((train_df[predictor_columns], times), axis=1)
test_predictors = np.array(test_df[predictor_columns])
test_pred_and_time = np.concatenate((test_predictors, np.zeros((len(test_predictors), 1)) + train_pred_and_time.Epsilon.max() + 1), axis=1)

In [16]:
model = WeightedEnsemble()
model.fit(np.array(train_pred_and_time), np.array(greeks_df['Alpha']))

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Multiple models in memory. This might lead to memory issues. Consider calling remove_models_from_memory()


In [17]:
probabilities = model.predict_proba(test_pred_and_time)
assert (model.classes_[0] == 'A')
probabilities = np.concatenate((probabilities[:,:1], np.sum(probabilities[:,1:], 1, keepdims=True)), axis=1)

In [18]:
probabilities

array([[0.49999998, 0.50000002],
       [0.49999998, 0.50000002],
       [0.49999998, 0.50000002],
       [0.49999998, 0.50000002],
       [0.49999998, 0.50000002]])

In [19]:
p0 = probabilities[:,:1]

In [20]:
p0

array([[0.49999998],
       [0.49999998],
       [0.49999998],
       [0.49999998],
       [0.49999998]])

In [21]:
p0 = probabilities[:,:1]
p0[p0 > 0.83] = 1
p0[p0 < 0.14] = 0

In [24]:
p0[p0 < 0.14]

array([], dtype=float64)

In [ ]:
submission = pd.DataFrame(test_df["Id"], columns=["Id"])
submission["class_0"] = p0
submission["class_1"] = 1 - p0
submission.to_csv('submission.csv', index=False)

In [ ]:
submission_df = pd.read_csv('submission.csv')
submission_df